<a href="https://colab.research.google.com/github/raghumriyer/colab/blob/master/LSTM_and_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
import os
import collections
import string
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [9]:
# Load the text file 'Alice in Wonderland' from the google drive location

from google.colab import drive
drive.mount('/content/drive')
file = "wonderland.txt"
filename = os.path.abspath("/content/drive/My Drive/EIP3/"+file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# load ascii text and covert to lowercase
raw_text = open(filename).read()
raw_text = raw_text.lower()

# Remove the punctuations from the text.
table = str.maketrans({key: None for key in string.punctuation})
raw_text = raw_text.translate(table) 

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

# load ascii text and covert to lowercase remove = dict.fromkeys(map(ord, '\n ' + string.punctuation)) table = str.maketrans({key: None for key in string.punctuation}) raw_text1 = open(filename).read() raw_text1 = raw_text1.lower() raw_text = raw_text1.translate(table)  # create mapping of unique chars to integers chars = sorted(list(set(raw_text))) char_to_int = dict((c, i) for i, c in enumerate(chars)) # summarize the loaded data n_chars = len(raw_text) n_vocab = len(chars) print("Total Characters: ", n_chars) print("Total Vocab: ", n_vocab) # prepare the dataset of input to output pairs encoded as integers seq_length = 100 dataX = []

for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]	
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

# Pad the sequnce of data with zeros
dataX = pad_sequences(dataX)

n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Characters:  154861
Total Vocab:  39
Total Patterns:  154761


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [12]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_2 (Dense)              (None, 38)                9766      
Total params: 799,270
Trainable params: 799,270
Non-trainable params: 0
_________________________________________________________________


In [13]:
# define the checkpoint
file="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

# save the file to the google drive
filepath = os.path.abspath("/content/drive/My Drive/EIP3/"+file)
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# Weight file already compiled
weightFile ="weights-improvement-22-0.7879-bigger.hdf5"

weightFilepath = os.path.abspath("/content/drive/My Drive/EIP3/"+weightFile)

# load the weight file
model.load_weights(weightFilepath)

# fit the model with epoch of 100 and batch size of 64
model.fit(X, y, epochs=10, batch_size=64, callbacks=callbacks_list)

Epoch 1/10
154761/154761 [==============================] - 674s 4ms/step - loss: 0.7989

Epoch 00001: loss improved from inf to 0.79888, saving model to /content/drive/My Drive/EIP3/weights-improvement-01-0.7989-bigger.hdf5
Epoch 2/10
154761/154761 [==============================] - 667s 4ms/step - loss: 2.3476

Epoch 00002: loss did not improve from 0.79888
Epoch 3/10
154761/154761 [==============================] - 664s 4ms/step - loss: 2.7001

Epoch 00003: loss did not improve from 0.79888
Epoch 4/10
154761/154761 [==============================] - 662s 4ms/step - loss: 2.6451

Epoch 00004: loss did not improve from 0.79888
Epoch 5/10
154761/154761 [==============================] - 658s 4ms/step - loss: 2.6168

Epoch 00005: loss did not improve from 0.79888
Epoch 6/10
154761/154761 [==============================] - 657s 4ms/step - loss: 2.5850

Epoch 00006: loss did not improve from 0.79888
Epoch 7/10
154761/154761 [==============================] - 656s 4ms/step - loss: 2.5450

